In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from homebrewedFunctions.functions import *
stack_dfs = pd.read_csv("StateGovFinances2005to2021.csv", index_col = ["State", "Year", "Format"], low_memory =False).sort_index()
panel_dfs_dict = {k:stack_dfs[stack_dfs.index.get_level_values(2)==k].reset_index().set_index(["State","Year"]).sort_index() for k in stack_dfs.index.get_level_values(2).unique()}
for key, df in panel_dfs_dict.items():
    for col in df.columns:
        if is_numeric_dtype(df[col]):
            df[col] = df[col].fillna(0)

        else:
            try:
                df[col] = pd.to_numeric(df[col], errors = "coerce").fillna(0).astype(int)
            except:
                pass
    df["1TOTAL INCOME"] = df["1INDIVIDUAL INCOME"].add(df["1CORPORATE INCOME"])
    df["1PROPERTY AND SPECIAL ASSESSMENTS"] = df[["1PROPERTY", "1SPECIAL ASSESSMENTS"]].sum(axis = 1)
    df["1DEFICIT"] = df["1EXPENDITURE"].sub(df["1GENERAL REVENUE"])
    # warning indicates that copying dataframe will defragment it;
    #  not sure if this actually fixes the problem
    ndf = df.copy()

SyntaxError: invalid syntax. Perhaps you forgot a comma? (functions.py, line 486)

In [ ]:
efnagdp = pd.read_csv("EFNAGDPTaxes.csv", parse_dates = ["Year"]).set_index(["State", "Year"]).sort_index()
efnagdp[["EFNA", "GDP"]]

EFNA           GDP
State Year                          
AK    1985-01-01  8.04           NaN
      1986-01-01   NaN           NaN
      1987-01-01   NaN           NaN
      1988-01-01   NaN           NaN
      1989-01-01   NaN           NaN
...                ...           ...
WY    2019-01-01  8.12  3.997140e+10
      2020-01-01  7.97  3.667550e+10
      2021-01-01   NaN  4.217620e+10
      2022-01-01   NaN  4.908060e+10
      2023-01-01   NaN  5.017160e+10

[1950 rows x 2 columns]

In [ ]:
map_keys = ["GENERAL REVENUE","GENERAL REVENUE FROM OWN SOURCES", "EXPENDITURE", "DEFICIT", 
        "FROM FEDERAL GOVERNMENT", "FROM STATE GOVERNMENT", "FROM LOCAL GOVERNMENTS" ,
        "TAXES", "OTHER TAXES", "CURRENT CHARGES", "OTHER CHARGES", 'INDIVIDUAL INCOME', 'CORPORATE INCOME', #'TOTAL INCOME',
        "PROPERTY", "SPECIAL ASSESSMENTS"]
        #"PROPERTY AND SPECIAL ASSESSMENTS",  
        # "SALES AND GROSS RECEIPTS", "GENERAL SALES", "MOTOR FUEL", "GAS SUPPLY", "EDUCATION", "HIGHER EDUCATION", "PUBLIC WELFARE"]	

keys = ["GENERAL REVENUE","GENERAL REVENUE FROM OWN SOURCES", "EXPENDITURE", "DEFICIT", 
        "FROM FEDERAL GOVERNMENT", "FROM STATE GOVERNMENT", "FROM LOCAL GOVERNMENTS" ,
        "TAXES", "OTHER TAXES", 'PROPERTY', 'SPECIAL ASSESSMENTS', 'PROPERTY AND SPECIAL ASSESSMENTS', 'SALES AND GROSS RECEIPTS', 'GENERAL SALES', 'SELECTIVE SALES', 'MOTOR FUEL', 'ALCOHOLIC BEVERAGE', 
        'TOBACCO PRODUCTS', 'PUBLIC UTILITIES', 'OTHER SELECTIVE SALES', 'INDIVIDUAL INCOME', 'CORPORATE INCOME', "TOTAL INCOME", 'MOTOR VEHICLE LICENSE', 
        'OTHER TAXES', 'CHARGES AND MISCELLANEOUS GENERAL  REVENUE', 'CURRENT CHARGES', 'EDUCATION', 'INSTITUTIONS  OF HIGHER EDUCATION', 
        'SCHOOL LUNCH SALES (GROSS)', 'HOSPITALS', 'HIGHWAYS', 'AIR TRANSPORTATION (AIRPORTS)', 'PARKING FACILITIES', 'SEA AND INLAND PORT FACILITIES', 
        'NATURAL RESOURCES', 'PARKS AND RECREATION', 'HOUSING AND COMMUNITY DEVELOPMENT', 'SEWERAGE', 'SOLID WASTE MANAGEMENT', 'OTHER CHARGES', 
        'MISCELLANEOUS GENERAL REVENUE', 'INTEREST EARNINGS',  'SALE OF PROPERTY', 'OTHER GENERAL REVENUE', 'UTILITY REVENUE', 
        'WATER SUPPLY', 'ELECTRIC POWER', 'GAS SUPPLY', 'TRANSIT', 'LIQUOR STORE REVENUE', 'INSURANCE TRUST REVENUE', 'UNEMPLOYMENT COMPENSATION', 
        'EMPLOYEE RETIREMENT', "WORKERS' COMPENSATION", 'OTHER INSURANCE TRUST REVENUE',"EDUCATION", "HIGHER EDUCATION", "PUBLIC WELFARE"]
keys = ["1" + k for k in keys]

In [ ]:
plot_dfs = {}
keys = ["GENERAL REVENUE","GENERAL REVENUE FROM OWN SOURCES", "EXPENDITURE", "DEFICIT", 
        "FROM FEDERAL GOVERNMENT", "FROM STATE GOVERNMENT", "FROM LOCAL GOVERNMENTS" ,
        "TAXES", "OTHER TAXES", "CURRENT CHARGES", "OTHER CHARGES", 'INDIVIDUAL INCOME', 'CORPORATE INCOME', 'TOTAL INCOME',
        "PROPERTY", "SPECIAL ASSESSMENTS", "PROPERTY AND SPECIAL ASSESSMENTS",  
        "SALES AND GROSS RECEIPTS", "GENERAL SALES", "MOTOR FUEL", "GAS SUPPLY", "EDUCATION", "HIGHER EDUCATION", "PUBLIC WELFARE"]	
def set_plot_dfs(panel_dfs_dict, keys, efnagdp):
    keys = ["1" + k for k in keys]
    for key in panel_dfs_dict.keys():
        plot_dfs[key] = {}
        plot_dfs[key]["Level"] = panel_dfs_dict[key][keys].mul(10**3).copy()
        plot_dfs[key]["Level"].rename(columns = {k:k.replace("1", "").title() for k in plot_dfs[key]["Level"].keys()}, inplace = True)
    #     plot_dfs[key]["Level"]["Total Income"] = plot_dfs[key]["Level"]["Individual Income"].add(plot_dfs[key]["Level"]["Corporate Income"])
    #     plot_dfs[key]["Level"]["Property and Special Assessments"] = plot_dfs[key]["Level"][["Property", "Special Assessments"]].sum(axis = 1)
    #     plot_dfs[key]["Level"]["Deficit"] = plot_dfs[key]["Level"]["Expenditure"].sub(plot_dfs[key]["Level"]["General Revenue"])
        plot_dfs[key]["Level"]["GDP"] = efnagdp["GDP"]
        plot_dfs[key]["Level"]["Population"] = efnagdp["Population"]
        plot_dfs[key]["Percent of General Revenue"] = plot_dfs[key]["Level"].apply(lambda x: pd.to_numeric(x).div(plot_dfs[key]["Level"]["General Revenue"]).mul(100))    
        plot_dfs[key]["Percent of GDP"] = plot_dfs[key]["Level"].apply(lambda x: pd.to_numeric(x).div(plot_dfs[key]["Level"]["GDP"]).mul(100))
        plot_dfs[key]["Value Per Capita"] = plot_dfs[key]["Level"].apply(lambda x: pd.to_numeric(x).div(plot_dfs[key]["Level"]["Population"]))    
        for key in plot_dfs.keys():
            plot_dfs[key]["Level"]["EFNA"] = efnagdp["EFNA"]
    return plot_dfs
plot_dfs = set_plot_dfs(panel_dfs_dict, keys, efnagdp)


In [ ]:
import os
for key in plot_dfs:
    try:
        os.mkdir(f"outputs/{key}")
    except:
        pass

In [ ]:
regions_df = pd.read_csv("USCensusRegions.csv")#.set_index("State")
# scatter_figs = {}
for key, p_dfs in plot_dfs.items():
    for p_dfskey, df in p_dfs.items():
        map_figs = {}
        create_scatter_dropdown(df, regions_df=regions_df,
                                filename = f"outputs/{key}/ScatterPlotsIncomePropertyAssessmentSalesFuelTaxesPctTotalRevenue{key}{p_dfskey}.html", 
                                show_fig = False)
        for name in df.keys():
            map_figs[name] = create_map(df.reset_index(), name, time_name = "Year")
        combined_map_fig = combine_map_figs(map_figs)
        combined_map_fig.write_html(f"outputs/{key}/MapPlotsByVariableAndYear{key}{p_dfskey}.html")
    # combined_scatter_fig = dict_of_figs_line_figs_to_dropdown_fig(scatter_figs,regions_df)



None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
['AK']
['AK']
['AL']
['AL']
['AR']
['AR']
['AZ']
['AZ']
['CA']
['CA']
['CO']
['CO']
['CT']
['CT']
['DC']
['DC']
['DE']
['DE']
['FL']
['FL']
['GA']
['GA']
['HI']
['HI']
['IA']
['IA']
['ID']
['ID']
['IL']
['IL']
['IN']
['IN']
['KS']
['KS']
['KY']
['KY']
['LA']
['LA']
['MA']
['MA']
['MD']
['MD']
['ME']
['ME']
['MI']
['MI']
['MN']
['MN']
['MO']
['MO']
['MS']
['MS']
['MT']
['MT']
['NC']
['NC']
['ND']
['ND']
['NE']
['NE']
['NH']
['NH']
['NJ']
['NJ']
['NM']
['NM']
['NV']
['NV']
['NY']
['NY']
['OH']
['OH']
['OK']
['OK']
['OR']
['OR']
['PA']
['PA']
['RI']
['RI']
['SC']
['SC']
['SD']
['SD']
['TN']
['TN']
['TX']
['TX']
['US']
['US']
['UT']
['UT']
['VA']
['VA']
['VT']
['VT']
['WA']
['WA']
['WI']
['WI']
['WV']
['WV']
['WY']
['WY']
['AK' 'AZ' 'CA' 'CO' 'HI' 'ID' 'MT' 'NM' 'NV' 'OR' 'UT' 'WA' 'WY']
['AK' 'AZ' 'CA' 'CO' 'HI' 'ID' 'MT' 'NM' 'NV' 'OR' 'UT'

In [ ]:
plot_dfs = {}
keys = ["GENERAL REVENUE","GENERAL REVENUE FROM OWN SOURCES", "EXPENDITURE", "DEFICIT", 
        "FROM FEDERAL GOVERNMENT", "FROM STATE GOVERNMENT", "FROM LOCAL GOVERNMENTS" , "INTERGOVERNMENTAL REVENUE",
        "TAXES", "OTHER TAXES", 'PROPERTY', 'SPECIAL ASSESSMENTS', 'PROPERTY AND SPECIAL ASSESSMENTS', 'SALES AND GROSS RECEIPTS', 'GENERAL SALES', 'SELECTIVE SALES', 'MOTOR FUEL', 'ALCOHOLIC BEVERAGE', 
        'TOBACCO PRODUCTS', 'PUBLIC UTILITIES', 'OTHER SELECTIVE SALES', 'INDIVIDUAL INCOME', 'CORPORATE INCOME', "TOTAL INCOME", 'MOTOR VEHICLE LICENSE', 
        'OTHER TAXES', 'CHARGES AND MISCELLANEOUS GENERAL  REVENUE', 'CURRENT CHARGES', 'EDUCATION', 'INSTITUTIONS  OF HIGHER EDUCATION', 
        'SCHOOL LUNCH SALES (GROSS)', 'HOSPITALS', 'HIGHWAYS', 'AIR TRANSPORTATION (AIRPORTS)', 'PARKING FACILITIES', 'SEA AND INLAND PORT FACILITIES', 
        'NATURAL RESOURCES', 'PARKS AND RECREATION', 'HOUSING AND COMMUNITY DEVELOPMENT', 'SEWERAGE', 'SOLID WASTE MANAGEMENT', 'OTHER CHARGES', 
        'MISCELLANEOUS GENERAL REVENUE', 'INTEREST EARNINGS',  'SALE OF PROPERTY', 'OTHER GENERAL REVENUE', 'UTILITY REVENUE', 
        'WATER SUPPLY', 'ELECTRIC POWER', 'GAS SUPPLY', 'TRANSIT', 'LIQUOR STORE REVENUE', 'INSURANCE TRUST REVENUE', 'UNEMPLOYMENT COMPENSATION', 
        'EMPLOYEE RETIREMENT', "WORKERS' COMPENSATION", 'OTHER INSURANCE TRUST REVENUE',"EDUCATION", "HIGHER EDUCATION", "PUBLIC WELFARE"]
plot_dfs = set_plot_dfs(panel_dfs_dict, keys, efnagdp)


In [ ]:
regions_df = pd.read_csv("USCensusRegions.csv")#.set_index("State")

for key, p_dfs in plot_dfs.items():
    figs = {k: line_dropdown(dataframe, regions_df) for k, dataframe in p_dfs.items()}
    fig = dict_of_figs_line_figs_to_dropdown_fig(figs, show_fig = False,
                                    use_sliders = True)
    # fig.show()
    fig.write_html(f"outputs/{key}/LinePlotsStateFinancesAsPercentRevenuePercentGDPAndPerCapitaFigs{key}.html")

# fig = line_dropdown(pct_df)
# fig.write_html("line_dropdown.html")

In [ ]:
start_year = 2005
gov_keys = ["Local government amount",'State & local government amount', 'State government amount']
states = plot_dfs[gov_keys[0]]["Level"].index.get_level_values("State").unique()
areas = {"Revenue Source by Government" : ["General Revenue From Own Sources", "From Federal Government", "From State Government", "From Local Governments"],
         
    #  Omit revenue sources according to distinction of subcomponents in revenue data 
    #  Some sources additionally needed to be removed as their values perfectly explained
    #  accounting that sums to greater than 100 percent
     
         "Taxes": ["Intergovernmental Revenue", 'Property', 'Sales And Gross Receipts', 
                   'Individual Income', 'Corporate Income', 'Motor Vehicle License', 'Other Taxes', 
                #    'Charges And Miscellaneous General  Revenue', 
                   'Current Charges', 
                #    'Education', 'Institutions  Of Higher Education', 'School Lunch Sales (Gross)',
                #      'Hospitals', 'Highways', 'Air Transportation (Airports)', 'Parking Facilities', 'Sea And Inland Port Facilities', 'Natural Resources',
                #       'Parks And Recreation', 'Housing And Community Development', 'Sewerage', 'Solid Waste Management', 'Other Charges', 
                    #   'Miscellaneous General Revenue', 
                      'Interest Earnings', 'Special Assessments', 'Sale Of Property', 'Other General Revenue']}
                    #   'Utility Revenue']}
                    #   'Water Supply', 'Electric Power', 'Gas Supply', 'Transit', 
                    # 'Liquor Store Revenue',
                #    'Insurance Trust Revenue',
                #    'Unemployment Compensation', 'Employee Retirement', "Workers' Compensation", 'Other Insurance Trust Revenue']}
figs = {}
for key in gov_keys:
    figs[key] = {}
    for form in ["Level", "Percent of General Revenue", "Percent of GDP", "Value Per Capita"]:
        figs[key][form] = {}
        df = plot_dfs[key][form].copy()
        df = df.reset_index().melt(id_vars=["State","Year"], 
            var_name="Name", 
            value_name="Value")
        df = df.set_index(["State", "Year"]).round(2)
        for components_group, components in areas.items():


            figs[key][form][components_group] = {}

            for state in states:
                plot_df = df.loc[state].reset_index()#.loc[start_year:].reset_index()
                plot_df = plot_df[plot_df["Name"].isin(components)].dropna()
                plot_df["Value"] = pd.to_numeric(plot_df["Value"])
                title_key = f"{components_group} {key}<br>{form}<br>{state}"
                px_fig = px.area(
                    plot_df, x="Year", y="Value", color="Name", title = title_key)

                figs[key][form][components_group][state] = px_fig
            figs[key][form][components_group]  = dict_of_figs_to_dropdown_fig(figs[key][form][components_group] , show_fig = False,
                                        use_sliders = True)

            figs[key][form][components_group].write_html(f"outputs/{key}/AreaPlots{key}{form}{components_group}Figs.html")


In [ ]:
regions_df["Region"].unique()
regions_df["Division"].unique()


array(['Pacific', 'East South Central', 'West South Central', 'Mountain',
       'New England', 'South Atlantic', 'West North Central',
       'East North Central', 'Middle Atlantic'], dtype=object)